# Recording live data

This code allows you to record live data from the [FinalSpark LiveMEA webview](https://livemea.finalspark.com/live) and save it to an HDF5 file.<br>
It provides both an API and a CLI for ease of use.

The code is available [at the following link](https://github.com/FinalSpark-np/LiveMEA).

```{note}
The code is usable even if you are not a NeuroPlatform user, and is freely available for anyone to use.<br>
Please reuse the provided code in your own projects and modify it as needed; the current implementation merely provides a simple way to record data from the live view.
```

## Data specification

The live view gives, roughly every second, 4096 samples of data from 32 electrodes.

In the following code, each "duration" unit corresponds to 4096 values; the sampling frequency is 30kHz downsampled via wavelet transform by a factor of 8.

**Therefore, a duration of 1 corresponds in reality to ~1.09 seconds of data.**

Expect a **disk usage of around 1.04 MB**, and an **average wait time of 1.09 seconds per second of recording**.

## Installation

We recommend using a virtual environment or a conda environment to install the required packages.
For anyone unfamiliar with or new to Python, we recommend installing [miniconda](https://docs.anaconda.com/miniconda/) to create a new environment.

1. Clone the repository:

    ```sh
    git clone https://github.com/FinalSpark-np/LiveMEA.git
    cd live-mea
    ```

````{margin}
```{hint}
To use the libraries you installed, you will have to activate the environment each time you open a new terminal.
```
````

2. Create and activate a conda environment:

    ```sh
    conda create --name livemea-env python=3.11
    conda activate livemea-env
    ```

3. Install the required packages:

    ```sh
    pip install -r requirements.txt
    ```

## API Usage

```{caution}
Using the `record()` method in Datalore/Jupyter will not work.
You will have to use `await record_async()` instead.
See the [Example](utils:livemea-record-example) below for more details.
```

### LiveMEA Class

#### Initialization

```python
from MEA_live import LiveMEA
from pathlib import Path

live_mea = LiveMEA(save_path: str | Path, recording_duration: int = 5, mea_id: int = 1)
```

- `save_path`
  (str | Path): Path to save the HDF5 file.
- `recording_duration`
 (int, optional): Duration of recording in "seconds" (chunks of 4096 samples).<br>Defaults to 5.
- `mea_id`
  (int, optional): MEA ID to use. Defaults to 1, must be between 1 and 4.

#### Methods

````{margin}
```{note}
The data is saved to the h5 automatically after the recording is complete.
```
````

- `record()` : Starts recording live data and saves it to the specified HDF5 file.
- `record_async()` : Starts recording live data asynchronously and saves it to the specified HDF5 file. Used within Datalore/Jupyter notebooks.
- `plot_data(h5f_path: str | Path)`: Plots data from an HDF5 file. Requires Matplotlib as an additional dependency.

(utils:livemea-record-example)=
### Example


In [ ]:
from MEA_live import LiveMEA

# Initialize the LiveMEA instance
live_mea = LiveMEA(save_path="live_data.h5", recording_duration=10, mea_id=1)

# Record data
# IF YOU ARE USING A PYTHON SCRIPT :
live_mea.record()
# IF YOU ARE USING DATALORE/JUPYTER :
data = await live_mea.record_async()

# Plot recorded data
LiveMEA.plot_data("live_data.h5")


## CLI Usage

You can also use a command-line interface (CLI) to record live MEA data.

### Command

From the repository root directory, run the following command:

```sh
python live-mea -d <duration> -p <path> -m <MEA>
```

### Arguments

- `-d`, `--duration` (int, optional): Duration of the recording in seconds. Defaults to 5.
- `-p`, `--path` (str, optional): Path to save the recorded data. Defaults to "live_data.h5".
- `-m`, `--MEA` (int, optional): MEA ID to record data from. Defaults to 1.

### Example

```sh
python live-mea -d 10 -p "live_data.h5" -m 1
```

This command will record live MEA data for 10 seconds and save it to `live_data.h5` using MEA 1.

```{note}
The CLI works both in DataLore and Jupyter notebooks.
Simply use `!` before the command within your notebook cell.
```

## Accessing the saved data

Each time point is saved under its own group in the HDF5 file. The data for each electrode is then saved as a dataset within the group.

The structure of the HDF5 file is as follows:

```
live_data.h5
├── timestamp_0
│   ├── electrode_0
│   ├── electrode_1
│   ├── ...
│   ├── electrode_31
├── timestamp_1
│   ├── ...
│   ├── electrode_31
├── ...
```

You can access the saved data using the `h5py` library in Python.

In [ ]:
import h5py

# Open the HDF5 file
with h5py.File("live_data.h5", "r") as f:
    # List all timestamps
    timestamps = list(f.keys())
    print("Timestamps:", timestamps)

    # Access data for a specific timestamp
    timestamp = timestamps[0]
    data_group = f[timestamp]

    # List all electrodes
    electrodes = list(data_group.keys())
    print("Electrodes:", electrodes)

    # Access data for a specific electrode
    electrode_data = data_group["electrode_0"][:]
    print("Electrode 0 data:", electrode_data)

## Note regarding future updates

```{warning}
We provide the current code to obtain live data as-is, and we do not guarantee its maintenance or future updates.

**The provided code may stop working without notice**, as our infrastructure is still in development, and constantly evolving and changing.<br>
In the event that the code is non-functional, please reach out to us, e.g [by opening an issue on the repository](https://github.com/FinalSpark-np/LiveMEA/issues).

Of course, feel free to open issues or pull requests anytime if you encounter any problems or have suggestions for improvements.
```